# Análise exploratória de dados

In [1]:
import pandas as pd
from IPython.display import display, Markdown

In [3]:
df = pd.read_csv('../data/raw/diabetes_prediction_dataset.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


## 1. Descrição dos dados

In [4]:
null_counts = df.isnull().sum()

print(null_counts)

display(Markdown(
    "---\n"
    f"Sabemos assim que a não existem valores faltantes."))

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64


---
Sabemos assim que a não existem valores faltantes.

In [5]:
df.describe(include='all')

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000,100000.000000,100000.00000,100000.000000,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,3,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
top,Female,NaN,NaN,NaN,No Info,NaN,NaN,NaN,NaN
freq,58552,NaN,NaN,NaN,35816,NaN,NaN,NaN,NaN
mean,NaN,41.885856,0.07485,0.039420,NaN,27.320767,5.527507,138.058060,0.085000
std,NaN,22.516840,0.26315,0.194593,NaN,6.636783,1.070672,40.708136,0.278883
min,NaN,0.080000,0.00000,0.000000,NaN,10.010000,3.500000,80.000000,0.000000
25%,NaN,24.000000,0.00000,0.000000,NaN,23.630000,4.800000,100.000000,0.000000
50%,NaN,43.000000,0.00000,0.000000,NaN,27.320000,5.800000,140.000000,0.000000
75%,NaN,60.000000,0.00000,0.000000,NaN,29.580000,6.200000,159.000000,0.000000


Observações:

- Mulheres são a maioria no conjunto de dados;
- A idade média dos pacientes é de aproximadamente 42 anos;
- O paciente mais jovem tinha aproximadamente 1 mês de vida;
- O paciente mais velho tinha 80 anos.

---

Vamos verificar a necessidade de manter o valor "Other" da coluna "gender"

In [7]:
count = (df['gender'] == 'Other').sum() #conta a presença do valor "Other" na coluna "gender"
print(f"Número de linhas com 'other' na coluna 'gender': {count}")

Número de linhas com 'other' na coluna 'gender': 18


Sendo um valor irrisório comparado à quantidade de dados nesse dataset, removeremos os valores "Other" da coluna.

In [8]:
df = df[df['gender'] != 'Other']

---

Agora verificaremos a presença de linhas duplicadas. Considerando a variabilidade em casas decimais de algumas variáveis, podemos assumir que linhas com os mesmos valores se referem ao mesmo paciente.

In [10]:
duplicatas: int = df.duplicated().sum()
print(f"Número de duplicatas: {duplicatas}")
df = df.drop_duplicates()

Número de duplicatas: 3854


## 2. Perguntas de partida e hipóteses

## 3. Insights